# S3W8D1: 统一接口封装 (The Universal Adapter)


### 📂 1. 今日工作区准备

请在终端执行以下命令，创建目录并启动 Notebook：

```bash
mkdir -p stage3/week8/data
# 建议的文件名：
touch stage3/week8/S3W8D1_LLM_Interface_Basics.ipynb
```

**文件名确认**：`S3W8D1_LLM_Interface_Basics.ipynb`

接下来，请打开这个 Notebook，我们将分三步完成今天的任务。

-----

### 🧪 2. Notebook 实验环节 (交互式学习)

#### **Step 1: 环境配置与密钥管理**

在 Notebook 的第一个 Cell 中，我们需要安装依赖并设置环境变量。
**注意**：为了防止密钥泄露到 Git 仓库，**永远不要**把 API Key 硬编码在代码里，而是使用 `.env` 文件。

**操作：**

1.  在项目根目录（`project_root/`）新建一个 `.env` 文件：
    ```text
    # .env 文件内容 (替换为你申请的 Key，建议用 DeepSeek 或 Moonshot)
    LLM_API_KEY=sk-xxxxxxxxxxxxxxxxxxxx
    LLM_BASE_URL=https://api.deepseek.com
    # 或者 Moonshot: https://api.moonshot.cn/v1
    ```
2.  在 Notebook 中运行：

<!-- end list -->

```python
# Cell 1
!pip install openai python-dotenv

import os
import sys
from dotenv import load_dotenv

# 加载 .env 环境变量
load_dotenv()

# 验证加载是否成功 (不要打印出完整的 Key，只打印前几位)
api_key = os.getenv("LLM_API_KEY")
base_url = os.getenv("LLM_BASE_URL")

print(f"Base URL: {base_url}")
print(f"API Key loaded: {'Yes' if api_key else 'No'} ({api_key[:4]}...)")
```

#### **Step 2: 编写与测试 LLMClient 类**

在 Notebook 中定义类，方便随时修改调试。我们需要处理 **网络错误** 和 **参数配置**。

```python
# Cell 2
from openai import OpenAI
from typing import Optional

class LLMClient:
    def __init__(self, 
                 api_key: Optional[str] = None, 
                 base_url: Optional[str] = None, 
                 model: str = "deepseek-chat"): # 根据你的服务商修改默认模型名
        
        self.api_key = api_key or os.getenv("LLM_API_KEY")
        self.base_url = base_url or os.getenv("LLM_BASE_URL")
        self.model = model
        
        if not self.api_key:
            raise ValueError("API Key 未找到，请检查 .env 文件或参数传入")

        # 初始化 OpenAI SDK
        self.client = OpenAI(api_key=self.api_key, base_url=self.base_url)

    def generate(self, prompt: str, system_prompt: str = "You are a helpful assistant.", temperature: float = 0.7) -> str:
        """
        生成回复
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=512 # 防止生成过长
            )
            return response.choices[0].message.content.strip()
        
        except Exception as e:
            # 面试加分点：生产环境中这里应该接入日志系统 (logging)
            print(f"[Error]生成失败: {e}")
            return "Error: 模型调用失败，请检查网络或Key。"

# 实例化测试
client = LLMClient()
print("测试连接...")
print(client.generate("你好，请做个自我介绍。"))
```

#### **Step 3: Temperature 实验 (面试核心考点)**

这是今天的重头戏。我们要直观感受 `Temperature` 对输出分布的影响。

> **医学图像类比**：
>
>   * **Temperature -\> 0**: 像是把图像分割的 **二值化阈值 (Thresholding)** 卡得很死。稍微有点不确定的像素直接丢弃，只保留概率最高的区域。结果非常锐利、确定，但可能丢失边缘细节。
>   * **Temperature -\> High**: 像是引入了 **高斯噪声**。虽然可能捕捉到更多潜在特征，但也可能把噪点当成病灶（产生幻觉）。

在 Notebook 中运行以下对比代码：

```python
# Cell 3
prompt = "请用3个词形容春天。"

print("--- Temperature = 0.1 (严谨/确定) ---")
for _ in range(3):
    print(client.generate(prompt, temperature=0.1))

print("\n--- Temperature = 0.9 (发散/创造) ---")
for _ in range(3):
    print(client.generate(prompt, temperature=0.9))
    
print("\n--- Temperature = 1.8 (极端/混乱) ---")
for _ in range(3):
    print(client.generate(prompt, temperature=1.8))
```

-----

### 🧠 3. 深度原理 (面试官视角)

在你运行完上述代码后，请仔细阅读这一段，这是你能否过面试的关键。

#### **Q1: Temperature 数学原理**

面试官问：“为什么 Temperature 变高，输出就变乱了？”

**回答逻辑**：
LLM 输出的本质是词表中每个 token 的 Logits（未归一化的分数）。Softmax 函数将 Logits 转化为概率。加入 Temperature ($T$) 后的公式为：

$$P_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}$$

  * $z_i$ 是第 $i$ 个词的 Logit。
  * **当 $T \to 0$ (低温)**：
      * $z_i / T$ 变得非常大（如果 $z_i$ 是正数）或非常小。
      * 指数运算 $\exp$ 会极度放大最大值和其他值的差距。
      * 结果：概率分布变成 **One-hot (尖峰)**，模型只选概率最大的那个词。**每次输出都一样。**
  * **当 $T \to \infty$ (高温)**：
      * $z_i / T \to 0$。
      * $\exp(0) = 1$。
      * 所有词的概率都趋近于相等（均匀分布）。
      * 结果：模型开始随机抽样，甚至选出原本概率极低的生僻词（胡言乱语）。

#### **Q2: 无状态 (Stateless)**

面试官问：“OpenAI API 是 Stateless 的，这对开发有什么影响？”

**回答逻辑**：

  * **含义**：服务器不保存任何上下文。你发给它“你好”，它回“你好”。你再发“我是谁？”，它不知道你是谁，因为它忘了上一轮对话。
  * **开发影响**：我们必须在客户端维护 **History (对话历史)**。每次请求时，都要把 `[SystemPrompt, User:你好, AI:你好, User:我是谁]` 这一整串 List 发给 API。
  * **代价**：Token 消耗随着对话轮数增加而线性（甚至二次方）增长。

-----

### 🏗️ 4. 工程封装 (Refactoring)

Notebook 跑通后，请把 `LLMClient` 类的代码正式写入工程文件。

**操作：**
把 Notebook 中的 `LLMClient` 类代码复制并粘贴到 `src/llm/client.py` 中。确保包含 `import os`, `dotenv` 等引用。

-----

### ✅ 5. 你的下一步

完成以上步骤后，请在 Notebook 结尾运行一次从 `src` 导入的测试，确保封装成功：

```python
# Cell 4
# 重新加载模块 (如果修改了 .py 文件需要这两行)
%load_ext autoreload
%autoreload 2

from src.llm.client import LLMClient

# 最终验证
final_client = LLMClient()
print(final_client.generate("测试 src 模块封装是否成功"))
```

**任务完成后，请回复“W8D1 笔记已完成，封装通过”，我们将进入 Day 2 的 Prompt Engineering 环节！**